In [82]:
import pymysql
import pandas as pd
import mysql.connector

In [83]:
dbConnection = mysql.connector.connect(
  host="localhost",
  port="3307",
  user="root",
  passwd="",
  database="consultation"
)

In [84]:
dbConnection1 = mysql.connector.connect(
  host="localhost",
  port="3307",
  user="root",
  passwd="",
  database="client"
)

In [85]:
dbConnection2 = mysql.connector.connect(
  host="localhost",
  port="3307",
  user="root",
  passwd="",
  database="facility"
)

In [86]:
data = pd.read_sql("select art_number,person_id,visit_type,tb_status,state from art a join art_current_status c on a.art_id = c.art_id join art_visit d on a.art_id=d.art_id", dbConnection)

InterfaceError: Failed parsing OK packet.

In [6]:
ArtID = pd.read_sql("select art_id,person_id from art",dbConnection)

In [7]:
dataWithArtID = data.merge(ArtID, on="person_id")

In [8]:
visitDate = pd.read_sql("select art_id,state,date from art_current_status",dbConnection)

In [9]:
DatedData = pd.merge(dataWithArtID,visitDate, how='left',left_on=['art_id','state'],right_on=['art_id','state'])

In [10]:
client = pd.read_sql("select person_id,birthdate,sex from person", dbConnection1);

In [11]:
merged = DatedData.merge(client, on="person_id")

In [12]:
trimmedArtNumbers = data['art_number'].str[:8]

In [13]:
out = trimmedArtNumbers.value_counts().index

In [43]:
s = out[0].replace('-','')

In [44]:
facilityConfig = pd.read_sql("select * from facility a join district b on a.district_id = b.district_id join province c on a.province_id = c.province_id",dbConnection2)

In [45]:
facilityConfig.drop(['district_id','province_id','dhis_code','longitude','latitude'], axis=1,inplace=True)

In [46]:
facilityConf = facilityConfig[facilityConfig['facility_id'].str.contains(s)]

In [47]:
province = facilityConf['facility_id'].str[2:4]

In [48]:
district = facilityConf['facility_id'].str[4:6]

In [49]:
faci = facilityConf['facility_id'].str[6:]

In [50]:
facilityConf['FacilityCode'] = faci

/home/ignatious/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [51]:
facilityConf['DistrictCode'] = district

/home/ignatious/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [52]:
facilityConf['ProvinceCode'] = province

/home/ignatious/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [53]:
facilityConf.columns.values[1] = "FacilityName"

In [54]:
facilityConf.columns.values[2] = "DistrictName"

In [55]:
facilityConf.columns.values[3] = "ProvinceName"

In [56]:
facilityConf

,facility_id,FacilityName,DistrictName,ProvinceName,FacilityCode,DistrictCode,ProvinceCode
589,ZW030325,Borera,Hwedza District,Mashonaland East Province,25,03,03


In [58]:
facility = facilityConf

In [59]:
a = facility['FacilityName'].str[:45].value_counts().index
b = a[0]

In [60]:
c = facility['FacilityCode'].str[:20].value_counts().index
d = c[0]

In [61]:
e = facility['DistrictCode'].str[:20].value_counts().index
f = e[0]

In [62]:
g = facility['ProvinceCode'].str[:20].value_counts().index
h = g[0]

In [63]:
i = facility['DistrictName'].str[:45].value_counts().index
j = i[0]

In [64]:
k = facility['ProvinceName'].str[:45].value_counts().index
l = k[0]

In [65]:
merged.drop(['person_id','art_id'], axis=1,inplace=True)

In [66]:
clearMerged = merged.assign(FacilityName=b,DistrictName=j,ProvinceName=l,FacilityCode=d,DistrictCode=f,ProvinceCode=h)

In [67]:
clearMerged.columns.values[3] = "art_status"

In [68]:
clearMerged.columns.values[4] = "visit_date"

In [69]:
clearMerged.head()

,art_number,visit_type,tb_status,art_status,visit_date,birthdate,sex,FacilityName,DistrictName,ProvinceName,FacilityCode,DistrictCode,ProvinceCode
0,03-03-25-2016-A-00018,Present self,Screened and has no signs,CONTINUE,2016-05-27,1980-06-11,FEMALE,Borera,Hwedza District,Mashonaland East Province,25,03,03
1,03-03-25-2016-A-00018,Present self,Screened and has no signs,CONTINUE,2016-08-05,1980-06-11,FEMALE,Borera,Hwedza District,Mashonaland East Province,25,03,03
2,03-03-25-2016-A-00018,Present self,Screened and has no signs,CONTINUE,2016-05-27,1980-06-11,FEMALE,Borera,Hwedza District,Mashonaland East Province,25,03,03
3,03-03-25-2016-A-00018,Present self,Screened and has no signs,CONTINUE,2016-08-05,1980-06-11,FEMALE,Borera,Hwedza District,Mashonaland East Province,25,03,03
4,03-03-25-2016-A-00018,Present self,Screened and has no signs,CONTINUE,2016-05-27,1980-06-11,FEMALE,Borera,Hwedza District,Mashonaland East Province,25,03,03


In [70]:
clearMerged.to_csv('/home/ignatious/Documents/ehr/zdatasets/%s.csv'%b, encoding='utf-8', index=False)

In [71]:
dbConnection.close()
dbConnection1.close()
dbConnection2.close()